**O conjunto de dados utilizado para criação desse modelo pode ser baixado em:** https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews/data

In [60]:
!pip install scipy

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors



In [6]:
avaliacoes = pd.read_csv('Books_rating.csv')
avaliacoes

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,NaN,NaN,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [10]:
avaliacoes = avaliacoes[['Title', 'User_id', 'review/score']]

In [11]:
avaliacoes.head()

,Title,User_id,review/score
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
2,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0


In [13]:
avaliacoes = avaliacoes.rename(columns={'Title' : 'titulo', 'User_id': 'id_usuario', 'review/score': 'avaliacao'})

In [14]:
avaliacoes

,titulo,id_usuario,avaliacao
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
2,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0
...,...,...,...
2999995,The Idea of History,NaN,4.0
2999996,The Idea of History,A1SMUB9ASL5L9Y,4.0
2999997,The Idea of History,A2AQMEKZKK5EE4,4.0
2999998,The Idea of History,A18SQGYBKS852K,5.0


In [16]:
avaliacoes.isna().sum()

titulo           208
id_usuario    561787
avaliacao          0
dtype: int64

In [17]:
avaliacoes.dropna(inplace=True)

In [19]:
avaliacoes.isna().sum()

titulo        0
id_usuario    0
avaliacao     0
dtype: int64

**Vamos considerar somente os usuários que fizeram mais de 100 avaliações**

In [23]:
avaliacoes['id_usuario'].value_counts()

A14OJS0VWMOSWO    5795
AFVQZQ8PW0L       3606
A1D2C0WDCSHUWZ    3146
AHD101501WCN1     1995
A1X8VZWTOG8IS6    1804
                  ... 
A2KGFK4UXJKK4O       1
A3REC6EA2Q5HCY       1
A1MC2GMXMP1GP8       1
APBF649GF6YTG        1
A18SQGYBKS852K       1
Name: id_usuario, Length: 1008961, dtype: int64

In [29]:
qt_avaliacoes = avaliacoes['id_usuario'].value_counts() > 100
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

(1013,)

In [30]:
y

Index(['A14OJS0VWMOSWO', 'AFVQZQ8PW0L', 'A1D2C0WDCSHUWZ', 'AHD101501WCN1',
       'A1X8VZWTOG8IS6', 'A1K1JW1C5CUSUZ', 'A20EEWWSFMZ1PN', 'A1S3C5OFU508P3',
       'A1N1YEMTI9DJ86', 'A2OJW07GQRNJUT',
       ...
       'A2Z9XBCYT3AHHU', 'AI9AFMD6OVGUG', 'A23SB6VGGB9E8U', 'A9MAUO67ABVOB',
       'A89TL80GBGNUK', 'A1P2XYD265YE21', 'A126JVOAOOR42V', 'A1TPW86OHXTXFC',
       'A3VO648HFK2LZD', 'A3GHSD9VY8XS4Q'],
      dtype='object', length=1013)

In [32]:
avaliacoes = avaliacoes[avaliacoes['id_usuario'].isin(y)]

In [33]:
avaliacoes

,titulo,id_usuario,avaliacao
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
6,Dr. Seuss: American Icon,A14OJS0VWMOSWO,5.0
59,Rising Sons and Daughters: Life Among Japan's ...,A3NIQK6ZLYEP1L,4.0
61,Muslim Women's Choices: Religious Belief and S...,ATDE9JYCPI0L1,2.0
140,Eyewitness Travel Guide to Europe,A281NPSIMI1C2R,5.0
...,...,...,...
2999949,Very Bad Deaths: Library Edition,AFVQZQ8PW0L,5.0
2999960,My Life,A9Q28YTLYREO7,5.0
2999971,My Life,A22LX6529JJ166,5.0
2999979,"Old Quebec,: The fortress of New France,",A32ZKBXJJ45BRY,3.0


**Vamos considerar somente os filmes que possuem mais 100 de avaliações**

In [35]:
avaliacoes['titulo'].value_counts()

Pride and Prejudice                                            1429
Wuthering Heights                                              1040
Persuasion                                                      884
The Hobbit                                                      872
The Picture of Dorian Gray                                      756
                                                               ... 
The Pirate's Jewel                                                1
The Great Wave: Price Revolutions and the Rhythm of History       1
Baby Whisperer Solves All Your Problems                           1
On The Edge                                                       1
The Idea of History                                               1
Name: titulo, Length: 62369, dtype: int64

In [39]:
qt_filmes = avaliacoes['titulo'].value_counts() > 200
x = qt_filmes[qt_filmes].index
x.shape

(37,)

In [40]:
avaliacoes = avaliacoes[avaliacoes['titulo'].isin(x)]

In [41]:
avaliacoes

,titulo,id_usuario,avaliacao
1739,The Scarlet Letter A Romance,A1YB29BIDULEYE,5.0
1747,The Scarlet Letter A Romance,AUM3YMZ0YRJE0,5.0
1761,The Scarlet Letter A Romance,A1FEEFAK5Y45M2,1.0
1789,The Scarlet Letter A Romance,A1JIPFV4OL520T,5.0
1807,The Scarlet Letter A Romance,AMKZHBOK7VMQR,5.0
...,...,...,...
2999633,Jane Eyre,A1D2C0WDCSHUWZ,5.0
2999638,Jane Eyre,AHUT55E980RDR,4.0
2999641,Jane Eyre,A1T17LMQABMBN5,5.0
2999646,Jane Eyre,AJQ1S39GZBKUG,5.0


In [42]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15153 entries, 1739 to 2999664
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   titulo      15153 non-null  object 
 1   id_usuario  15153 non-null  object 
 2   avaliacao   15153 non-null  float64
dtypes: float64(1), object(2)
memory usage: 473.5+ KB


**Vamos transformar cada id de usuário em uma coluna**

In [43]:
avaliacoes_pivot = avaliacoes.pivot_table(columns = 'id_usuario', index = 'titulo' , values = 'avaliacao')

In [44]:
avaliacoes_pivot

id_usuario,A10T0OW97SFBB,A10VOEBL5S337W,A114YQ7ZT9Y1W5,A11B61QBGHLQDN,A11DCTGTPS7M0C,A11M98R135HMSY,A1241U6QCSX5YJ,A126JVOAOOR42V,A126KX6FVI4T66,A1288XI5LO0J76,...,AWS5NBJCYLY7A,AXQ8T2D9IT6UG,AXSH6O4OP5L4M,AXVXIWJ6IU6KA,AYCJSA9HR7TKO,AYFNOHE3IQQ4G,AYPOMVBFYBVWE,AYT4FJYVCHYLE,AZSNQ9FYC5SQO,AZSV99SDJC242
titulo,,,,,,,,,,,,,,,,,,,,,
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Connecticut Yankee in King Arthur's Court,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Farewell to Arms,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Tree Grows in Brooklyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
An inquiry into the nature and causes of the wealth of nations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Atlas Shrugged,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brave New World,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crime and Punishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
East of Eden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
avaliacoes_pivot.fillna(0, inplace=True)

In [47]:
avaliacoes_pivot.head()

id_usuario,A10T0OW97SFBB,A10VOEBL5S337W,A114YQ7ZT9Y1W5,A11B61QBGHLQDN,A11DCTGTPS7M0C,A11M98R135HMSY,A1241U6QCSX5YJ,A126JVOAOOR42V,A126KX6FVI4T66,A1288XI5LO0J76,...,AWS5NBJCYLY7A,AXQ8T2D9IT6UG,AXSH6O4OP5L4M,AXVXIWJ6IU6KA,AYCJSA9HR7TKO,AYFNOHE3IQQ4G,AYPOMVBFYBVWE,AYT4FJYVCHYLE,AZSNQ9FYC5SQO,AZSV99SDJC242
titulo,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Connecticut Yankee in King Arthur's Court,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Farewell to Arms,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Tree Grows in Brooklyn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
An inquiry into the nature and causes of the wealth of nations,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
avaliacoes_sparse = csr_matrix(avaliacoes_pivot)

In [59]:
type(avaliacoes_sparse)

scipy.sparse._csr.csr_matrix

In [62]:
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(avaliacoes_sparse)

NearestNeighbors(algorithm='brute')

In [63]:
#testando o modelo

distances, sugestions = modelo.kneighbors(avaliacoes_pivot.filter(items = ['Pride and Prejudice'], axis=0).values.reshape(1,-1))


for i in range(len(sugestions)):
    print(avaliacoes_pivot.index[sugestions[i]])

Index(['Pride and Prejudice', 'Sense and Sensibility', 'Emma', 'Persuasion',
       'Little Women'],
      dtype='object', name='titulo')
